# Prepare Data

In [4]:
import pandas as pd
yellow_tripdata = pd.read_parquet('yellow_tripdata_2009-02.parquet')

In [5]:
yellow_tripdata

,vendor_name,Trip_Pickup_DateTime,Trip_Dropoff_DateTime,Passenger_Count,Trip_Distance,Start_Lon,Start_Lat,Rate_Code,store_and_forward,End_Lon,End_Lat,Payment_Type,Fare_Amt,surcharge,mta_tax,Tip_Amt,Tolls_Amt,Total_Amt
0,DDS,2009-02-03 08:25:00,2009-02-03 08:33:39,1,1.60,-73.992768,40.758325,NaN,NaN,-73.994710,40.739723,CASH,6.9,0.0,NaN,0.00,0.0,6.90
1,VTS,2009-02-28 00:26:00,2009-02-28 00:40:00,5,3.05,0.000000,0.000000,NaN,NaN,0.000000,0.000000,CASH,10.5,0.5,NaN,0.00,0.0,11.00
2,DDS,2009-02-22 00:39:23,2009-02-22 00:45:52,1,1.50,-73.137393,41.366138,NaN,NaN,-73.137393,41.366138,CASH,5.7,0.5,NaN,0.00,0.0,6.20
3,VTS,2009-02-28 12:47:00,2009-02-28 12:54:00,1,1.42,-73.980458,40.748445,NaN,NaN,-73.996103,40.737095,CASH,6.1,0.0,NaN,0.00,0.0,6.10
4,DDS,2009-02-05 18:34:35,2009-02-05 18:43:26,1,1.50,-73.137393,41.366138,NaN,NaN,-73.137393,41.366138,CASH,6.9,1.0,NaN,0.00,0.0,7.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13380117,DDS,2009-02-01 17:09:40,2009-02-01 17:17:10,3,1.50,-73.970121,40.748764,NaN,NaN,-73.962825,40.766523,CASH,6.5,0.0,NaN,0.00,0.0,6.50
13380118,DDS,2009-02-03 13:13:43,2009-02-03 13:30:19,1,1.10,-73.985543,40.752639,NaN,NaN,-73.960740,40.759545,CASH,6.9,0.0,NaN,0.00,0.0,6.90
13380119,DDS,2009-02-01 15:51:00,2009-02-01 16:05:29,2,2.10,-73.975309,40.733122,NaN,NaN,-74.005796,40.733122,CREDIT,8.9,0.0,NaN,1.75,0.0,10.65
13380120,DDS,2009-02-07 01:33:43,2009-02-07 01:54:41,1,4.40,-73.972443,40.754235,NaN,NaN,-74.010250,40.719166,CASH,13.7,0.5,NaN,0.00,0.0,14.20


In [6]:
intresting_cols = ["Trip_Pickup_DateTime", "Start_Lon", "Start_Lat","Total_Amt"]

intresting_data = yellow_tripdata[intresting_cols].dropna()

# delete the rows with lon not in the range of [-180, 180] and lat not in the range of [-90, 90]
intresting_data = intresting_data[(intresting_data["Start_Lon"] >= -180) & (intresting_data["Start_Lon"] <= 180) & (intresting_data["Start_Lat"] >= -90) & (intresting_data["Start_Lat"] <= 90)]

min_start_lon = intresting_data["Start_Lon"].min()

max_start_lon = intresting_data["Start_Lon"].max()

min_start_lat = intresting_data["Start_Lat"].min()

max_start_lat = intresting_data["Start_Lat"].max()

#convert the date time to datetime object, and get the timestamp
intresting_data["Trip_Pickup_DateTime"] = pd.to_datetime(intresting_data["Trip_Pickup_DateTime"])

min_trip_pickup_datetime = intresting_data["Trip_Pickup_DateTime"].min().value

max_trip_pickup_datetime = intresting_data["Trip_Pickup_DateTime"].max().value

print(min_start_lon, max_start_lon, min_start_lat, max_start_lat, min_trip_pickup_datetime, max_trip_pickup_datetime)



-143.250622 1.216666 -0.052532 73.999865 1233446400000000000 1235865599000000000


In [19]:
intresting_data.min()

Trip_Pickup_DateTime    2009-02-01 00:00:00
Start_Lon                       -143.250622
Start_Lat                         -0.052532
Total_Amt                               2.5
dtype: object

In [7]:
# count the number of lontitude values that are less than -74.5 or greater than -73.5

intresting_data["Start_Lon"].loc[(intresting_data["Start_Lon"] < -90) | (intresting_data["Start_Lon"] > -50)].count()

197642

In [8]:
def get_grid_loc(time, lon, lat, min_time = min_trip_pickup_datetime, max_time = max_trip_pickup_datetime, min_lon = min_start_lon, max_lon = max_start_lon, min_lat = min_start_lat, max_lat = max_start_lat, grid_size = 100):
    time_loc = int((time.value - min_time)/(max_time - min_time+0.000001)*grid_size)
    lon_loc = int((lon - min_lon)/(max_lon - min_lon+0.000001)*grid_size)
    lat_loc = int((lat - min_lat)/(max_lat - min_lat+0.000001)*grid_size)
    time_loc = time_loc - 1 if time_loc != 0 else 0
    lon_loc = lon_loc - 1 if lon_loc != 0 else 0
    lat_loc = lat_loc - 1 if lat_loc != 0 else 0
    return (lon_loc, lat_loc, time_loc)
    

In [9]:
intresting_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13380112 entries, 0 to 13380121
Data columns (total 4 columns):
 #   Column                Dtype         
---  ------                -----         
 0   Trip_Pickup_DateTime  datetime64[ns]
 1   Start_Lon             float64       
 2   Start_Lat             float64       
 3   Total_Amt             float64       
dtypes: datetime64[ns](1), float64(3)
memory usage: 510.4 MB


In [10]:
intresting_data

,Trip_Pickup_DateTime,Start_Lon,Start_Lat,Total_Amt
0,2009-02-03 08:25:00,-73.992768,40.758325,6.90
1,2009-02-28 00:26:00,0.000000,0.000000,11.00
2,2009-02-22 00:39:23,-73.137393,41.366138,6.20
3,2009-02-28 12:47:00,-73.980458,40.748445,6.10
4,2009-02-05 18:34:35,-73.137393,41.366138,7.90
...,...,...,...,...
13380117,2009-02-01 17:09:40,-73.970121,40.748764,6.50
13380118,2009-02-03 13:13:43,-73.985543,40.752639,6.90
13380119,2009-02-01 15:51:00,-73.975309,40.733122,10.65
13380120,2009-02-07 01:33:43,-73.972443,40.754235,14.20


In [11]:
from  tqdm import tqdm
grid = [[[[] for i in range(100)] for j in range(100)] for k in range(100)]

for index, row in tqdm(intresting_data.iterrows(),total=intresting_data.shape[0]):
    grid_loc = get_grid_loc(row["Trip_Pickup_DateTime"], row["Start_Lon"], row["Start_Lat"])
    grid[grid_loc[0]][grid_loc[1]][grid_loc[2]].append((row["Start_Lon"], row["Start_Lat"],row["Trip_Pickup_DateTime"].value, row["Total_Amt"]))

100%|██████████| 13380112/13380112 [08:38<00:00, 25797.21it/s]


In [12]:
# # multi threading
# import threading
# import os

# def write_to_grid(df, start_iloc, end_iloc,grid):
#     for index in tqdm(range(start_iloc, end_iloc)):
#         row = df.iloc[index]
#         grid_loc = get_grid_loc(row["Trip_Pickup_DateTime"], row["Start_Lon"], row["Start_Lat"])
#         grid[grid_loc[0]][grid_loc[1]][grid_loc[2]].append((row["Start_Lon"], row["Start_Lat"],row["Trip_Pickup_DateTime"].value, row["Total_Amt"]))

# cpu_count = os.cpu_count()
# test_grid = [[[[] for i in range(100)] for j in range(100)] for k in range(100)]
# for i in range(cpu_count):
#     start_index = intresting_data.shape[0]//cpu_count*i
#     end_index = intresting_data.shape[0]//cpu_count*(i+1) if i != cpu_count-1 else intresting_data.shape[0]
    
#     threading.Thread(target=write_to_grid, args=(intresting_data, start_index, end_index,test_grid)).start()

In [20]:
from datetime import datetime
def query_text_to_list(s):
    query_str_list = s.strip().split(",")
    low_x = float(query_str_list[0])
    up_x = float(query_str_list[1])
    low_y = float(query_str_list[2])
    up_y = float(query_str_list[3])
    low_time = int(datetime.strptime(query_str_list[4], "%Y-%m-%d %H:%M:%S").timestamp())*1000000000
    up_time = int(datetime.strptime(query_str_list[5], "%Y-%m-%d %H:%M:%S").timestamp())*1000000000
    print(query_str_list)
    print(low_time, up_time)
    return [low_x, up_x, low_y, up_y, low_time, up_time]
with open("queries.txt") as f:
    queries = f.readlines()
    struc_queries = list(map(query_text_to_list, queries))
    
    for query in struc_queries:
        low_x, up_x, low_y, up_y, low_time, up_time = query
        total = 0
        for data in tqdm(intresting_data.itertuples(),total=intresting_data.shape[0]):
            if low_x <= data.Start_Lon <= up_x and low_y <= data.Start_Lat <= up_y and low_time <= data.Trip_Pickup_DateTime.value <= up_time:
                total += data.Total_Amt
        print(total)

['-102', '-79.934', '38', '56.57', '2009-02-25 08:38:24', '2009-02-27 11:25:56']
1235522304000000000 1235705156000000000
['-85.8', '-82.1', '-39.1', '44.4', '2009-02-14 07:59:52', '2009-02-15 22:32:16']
1234569592000000000 1234708336000000000
['-78.7', '-73.969', '31.808', '40.761', '2009-02-14 02:57:22', '2009-02-27 11:08:06']
1234551442000000000 1235704086000000000
['-73.9983', '-73.994', '40.7356', '40.745', '2009-02-01 02:53:37', '2009-02-03 23:03:00']
1233428017000000000 1233673380000000000
['-73.9983', '-73.988', '40.7356', '40.746', '2009-02-01 02:53:37', '2009-02-06 16:30:00']
1233428017000000000 1233909000000000000
['-74.1', '-73.992', '40.742', '40.752', '2009-02-02 02:00:00', '2009-02-12 10:30:00']
1233511200000000000 1234405800000000000
['-76.1', '-73.211', '40.631', '41.711', '2009-02-02 02:00:00', '2009-02-15 10:30:00']
1233511200000000000 1234665000000000000
['-76.1', '-73.993', '40.631', '47.181', '2009-02-24 02:00:00', '2009-02-27 10:30:00']
1235412000000000000 1235701

100%|██████████| 13380112/13380112 [00:25<00:00, 517325.82it/s]


29.02


100%|██████████| 13380112/13380112 [00:17<00:00, 772932.92it/s] 


16.6


...

2210603.540000049


100%|██████████| 13380112/13380112 [00:22<00:00, 587786.84it/s]


64814503.07000895


100%|██████████| 13380112/13380112 [00:19<00:00, 678829.02it/s]

3784426.3000001134


In [13]:
with open("grid.txt", "w") as f:
    header = f"""{min_start_lon}, {max_start_lon}\n{min_start_lat}, {max_start_lat}\n{min_trip_pickup_datetime}, {max_trip_pickup_datetime}\n"""
    f.write(header)
    for i in range(100):
        for j in range(100):
            for k in range(100):
                f.write(f"({i},{j},{k})\n")
                total = sum([l[3] for l in grid[i][j][k]])
                f.write(f"{total}\n")
                for l in grid[i][j][k]:
                    f.write(f"{l[0]},{l[1]},{l[2]},{l[3]}\n")

# Parse Query

In [14]:
from datetime import datetime
def query_text_to_list(s):
    query_str_list = s.strip().split(",")
    low_x = float(query_str_list[0])
    up_x = float(query_str_list[1])
    low_y = float(query_str_list[2])
    up_y = float(query_str_list[3])
    low_time = int(datetime.strptime(query_str_list[4], "%Y-%m-%d %H:%M:%S").timestamp())*1000000000
    up_time = int(datetime.strptime(query_str_list[5], "%Y-%m-%d %H:%M:%S").timestamp())*1000000000
    return [low_x, up_x, low_y, up_y, low_time, up_time]
    

with open("queries.txt") as f:
    queries = f.readlines()
    struc_queries = list(map(query_text_to_list, queries))
print(struc_queries[:10])

[[-102.0, -79.934, 38.0, 56.57, 1235522304000000000, 1235705156000000000], [-85.8, -82.1, -39.1, 44.4, 1234569592000000000, 1234708336000000000], [-78.7, -73.969, 31.808, 40.761, 1234551442000000000, 1235704086000000000], [-73.9983, -73.994, 40.7356, 40.745, 1233428017000000000, 1233673380000000000], [-73.9983, -73.988, 40.7356, 40.746, 1233428017000000000, 1233909000000000000], [-74.1, -73.992, 40.742, 40.752, 1233511200000000000, 1234405800000000000], [-76.1, -73.211, 40.631, 41.711, 1233511200000000000, 1234665000000000000], [-76.1, -73.993, 40.631, 47.181, 1235412000000000000, 1235701800000000000]]


In [15]:
def get_grid_loc(time, lon, lat, min_time = min_trip_pickup_datetime, max_time = max_trip_pickup_datetime, min_lon = min_start_lon, max_lon = max_start_lon, min_lat = min_start_lat, max_lat = max_start_lat, grid_size = 100):
    time_loc = int((time.value - min_time)/(max_time - min_time+0.000001)*grid_size)
    lon_loc = int((lon - min_lon)/(max_lon - min_lon+0.000001)*grid_size)
    lat_loc = int((lat - min_lat)/(max_lat - min_lat+0.000001)*grid_size)
    time_loc = time_loc - 1 if time_loc != 0 else 0
    lon_loc = lon_loc - 1 if lon_loc != 0 else 0
    lat_loc = lat_loc - 1 if lat_loc != 0 else 0
    return (lon_loc, lat_loc, time_loc)

In [16]:
def calc_fraction(i,j,k, low_x, up_x, low_y, up_y, low_time, up_time):
    min_x = min_start_lon + (max_start_lon - min_start_lon)/100*i
    max_x = min_start_lon + (max_start_lon - min_start_lon)/100*(i+1)
    min_y = min_start_lat + (max_start_lat - min_start_lat)/100*j
    max_y = min_start_lat + (max_start_lat - min_start_lat)/100*(j+1)
    min_time = min_trip_pickup_datetime + (max_trip_pickup_datetime - min_trip_pickup_datetime)/100*k
    max_time = min_trip_pickup_datetime + (max_trip_pickup_datetime - min_trip_pickup_datetime)/100*(k+1)
    if min_x<low_x:
        x_per = (max_x - low_x)/(max_x - min_x)
    else:
        x_per = (up_x - min_x)/(max_x - min_x)
    if min_y<low_y:
        y_per = (max_y - low_y)/(max_y - min_y)
    else:
        y_per = (up_y - min_y)/(max_y - min_y)
    if min_time<low_time:
        time_per = (max_time - low_time)/(max_time - min_time)
    else:
        time_per = (up_time - min_time)/(max_time - min_time)
    return x_per*y_per*time_per

In [17]:
class Query:
    def __init__(self, low_x, up_x, low_y, up_y, low_time, up_time):
        grid_size = 100
        self.low_x = low_x
        self.up_x = up_x
        self.low_y = low_y
        self.up_y = up_y
        self.low_time = low_time
        self.up_time = up_time
        low_x_grid = int((self.low_time - min_trip_pickup_datetime)/(max_trip_pickup_datetime - min_trip_pickup_datetime+0.000001)*grid_size)
        up_x_grid = int((self.up_time - min_trip_pickup_datetime)/(max_trip_pickup_datetime - min_trip_pickup_datetime+0.000001)*grid_size)
        low_y_grid = int((self.low_x - min_start_lon)/(max_start_lon - min_start_lon+0.000001)*grid_size)
        up_y_grid = int((self.up_x - min_start_lon)/(max_start_lon - min_start_lon+0.000001)*grid_size)
        low_time_grid = int((self.low_y - min_start_lat)/(max_start_lat - min_start_lat+0.000001)*grid_size)
        up_time_grid = int((self.up_y - min_start_lat)/(max_start_lat - min_start_lat+0.000001)*grid_size)
        self.low_x_grid = low_x_grid - 1 if low_x_grid != 0 else 0
        self.up_x_grid = up_x_grid - 1 if up_x_grid != 0 else 0
        self.low_y_grid = low_y_grid - 1 if low_y_grid != 0 else 0
        self.up_y_grid = up_y_grid - 1 if up_y_grid != 0 else 0
        self.low_time_grid = low_time_grid - 1 if low_time_grid != 0 else 0
        self.up_time_grid = up_time_grid - 1 if up_time_grid != 0 else 0
        self.grid_total_amts = [[[sum([l[3] for l in grid[i][j][k]]) for i in range(100)] for j in range(100)] for k in range(100)]
    def get_low_x_grid(self):
        return self.low_x_grid
    def get_up_x_grid(self):
        return self.up_x_grid
    def get_low_y_grid(self):
        return self.low_y_grid
    def get_up_y_grid(self):
        return self.up_y_grid
    def get_low_time_grid(self):
        return self.low_time_grid
    def get_sum_grid_points_in_query(self,grid_points):
        # print(grid_points)
        sum = 0
        for grid_point in grid_points:
            if grid_point[0] >= self.low_x and grid_point[0] <= self.up_x and grid_point[1] >= self.low_y and grid_point[1] <= self.up_y and grid_point[2] >= self.low_time and grid_point[2] <= self.up_time:
                sum+=grid_point[3]
        return sum
    def exac_query(self):
        sum = 0
        for i in range(self.low_x_grid, self.up_x_grid+1):
            for j in range(self.low_y_grid, self.up_y_grid+1):
                for k in range(self.low_time_grid, self.up_time_grid+1):
                    if i == self.low_x_grid or i == self.up_x_grid or j == self.low_y_grid or j == self.up_y_grid or k == self.low_time_grid or k == self.up_time_grid:
                        sum+=self.get_sum_grid_points_in_query(grid[i][j][k])
                    else:
                        sum += self.grid_total_amts[i][j][k]
        return sum
    def approx_query(self):
        sum = 0
        for i in range(self.low_x_grid, self.up_x_grid+1):
            for j in range(self.low_y_grid, self.up_y_grid+1):
                for k in range(self.low_time_grid, self.up_time_grid+1):
                    if i == self.low_x_grid or i == self.up_x_grid or j == self.low_y_grid or j == self.up_y_grid or k == self.low_time_grid or k == self.up_time_grid:
                        sum += calc_fraction(i,j,k, self.low_x, self.up_x, self.low_y, self.up_y, self.low_time, self.up_time)*self.grid_total_amts[i][j][k]
                    else:
                        sum += self.grid_total_amts[i][j][k]
        return sum
    

        
        

In [18]:
from tqdm import tqdm
with open("exact_results.txt", "w") as f:
    for query in tqdm(struc_queries):
        query = Query(*query)
        f.write(str(query.exac_query()) + "\n")
        
with open("approx_results.txt", "w") as f:
    for query in tqdm(struc_queries):
        query = Query(*query)
        f.write(str(query.approx_query()) + "\n")
        

100%|██████████| 8/8 [00:15<00:00,  1.92s/it]
